## Business Problems:

## Business Problem Statement for Machine Learning
**How to segment the customers at Olist marketplace so we can divide customers based on their shopping behaviour?**
- Value	: Customer Segmentation
- Goals	: Divide customers based on their shopping behaviour

## Business Problem Statement for Analytics
**What kind of treatment for each cluster to increase retention rate customer?**
- Value	: Give treatment for each cluster
- Goals	: Increase retention rate customer


In [ ]:
# Data
import pandas as pd
import datetime as dt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Understanding:
- Data mengenai perusahaan *E-commerce* di Brazil, yaitu perusahaan *Olist*
- Data ini diambil dari tahun 2016 - 2018 dan disebarkan secara public di <a href = "https://www.kaggle.com/olistbr/brazilian-ecommerce">Kaggle</a> pada tahun 2018 bulan September
- Data-data yang ada berisi tentang transaksi *customer* dan *seller*
- Karena yang ingin dilakukan adalah *customer segmentation* maka hanya akan diambil data yang mengenai *customer* saja
- Ada 116584 data transaksi

### Merge Data:
Dikarenakan data yang berhubungan dengan transaksi *customer* terpisah-pisah, hal pertama yang harus dilakukan adalah mengabungkan data-data tersebut.

<p> <p>
<figure>
  <center><img src = "https://i.imgur.com/HRhd2Y0.png" alt ="Scheme Data Table" width = 700; height = 500></center>
  <figcaption align = "center">Gambar 1. <em>Scheme Data</em> </figcaption>
</figure>
<p> <p>

In [ ]:
# Read data yang dibutuhkan
olist_order = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_orders_dataset.csv')
olist_item = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_order_items_dataset.csv')
olist_payment = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_order_payments_dataset.csv')
olist_product = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_products_dataset.csv')
olist_customer = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_customers_dataset.csv')
olist_review = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/olist_order_reviews_dataset.csv')
olist_translate = pd.read_csv('/content/drive/MyDrive/Project Purwadhika/Dataset/product_category_name_translation.csv')

In [ ]:
# Merge data menjadi satu 
df_olist = pd.merge(olist_order, olist_item, on='order_id', how='left')
df_olist = pd.merge(df_olist, olist_product, on='product_id', how='inner')
df_olist = pd.merge(df_olist, olist_payment, on='order_id', how = 'left')
df_olist = pd.merge(df_olist, olist_review, on='order_id', how='left')
df_olist = pd.merge(df_olist, olist_customer, on='customer_id', how='right')
df_olist = pd.merge(df_olist, olist_translate, on='product_category_name', how='inner')

df_olist.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
0,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,1.0,credit_card,2.0,146.87,88b8b52d46df026a9d1ad2136a59b30b,4.0,NaN,NaN,2017-05-26 00:00:00,2017-05-30 22:34:40,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,office_furniture
1,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,moveis_escritorio,55.0,955.0,1.0,8267.0,52.0,52.0,17.0,1.0,credit_card,7.0,157.73,5ad6695d76ee186dc473c42706984d87,5.0,NaN,NaN,2018-06-15 00:00:00,2018-06-15 12:10:59,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,office_furniture
2,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,moveis_escritorio,48.0,1066.0,1.0,12160.0,56.0,51.0,28.0,1.0,credit_card,1.0,173.30,059a801bb31f6aab2266e672cab87bc5,5.0,NaN,NaN,2018-03-29 00:00:00,2018-04-02 18:36:47,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,office_furniture
3,997fd4b37386d10b57d4ac8cdec011a6,4c06b42fbf7b97ab10779cda5549cd1c,delivered,2018-07-08 09:59:19,2018-07-08 10:30:15,2018-07-25 15:12:00,2018-08-08 19:42:53,2018-08-16 00:00:00,1.0,8ed094bfe076c568f6bb10feada3f75d,7c67e1448b00f6e969d365cea6b010ab,2018-07-26 10:30:15,199.98,115.77,moveis_escritorio,43.0,1128.0,1.0,16133.0,63.0,59.0,25.0,1.0,credit_card,6.0,315.75,2518803238cabbf63564593e4a8fc52d,1.0,Nada de produto,"Ainda não recebi o produto. Sendo assim, não p...",2018-08-09 00:00:00,2018-08-14 11:40:54,07d190f123147d9e89d4b922543d7948,65075,sao luis,MA,office_furniture
4,b63fff1ef8b6201df68bb611ee41129e,8276de07ef25225d412b8462d73f8664,delivered,2017-04-02 22:15:27,2017-04-04 05:55:19,2017-04-13 16:28:06,2017-05-02 13:53:25,2017-05-08 00:00:00,1.0,aba86c093ccdbac75b09111d57e50004,7c67e1448b00f6e969d365cea6b010ab,2017-04-17 05:55:19,129.60,16.61,moveis_escritorio,41.0,154.0,1.0,2600.0,40.0,45.0,39.0,1.0,boleto,1.0,146.21,84467f1766775ba4c14fc25b3ca67905,4.0,NaN,"A crítica é à entrega, pois por se tratar dos ...",2017-05-03 00:00:00,2017-05-03 21:06:33,332cf4e83e16004ba7dca932ce82475b,90010,porto alegre,RS,office_furniture


In [ ]:
print("Jumlah row yang dimiliki: ", len(df_olist))

Jumlah row yang dimiliki:  116584


In [ ]:
df_olist.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value', 'review_id',
       'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name_english'],
      dtype='object')

### Drop columns:
Setelah penggabungan data dari beberapa tables, kami akan menghapus beberapa column yang tidak ada kaitannya dengan transaksi *behavior* dari *customer* 

In [ ]:
df_olist_clean = df_olist.drop(columns=['order_status', 'order_approved_at', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 
                                  'order_estimated_delivery_date', 'seller_id', 'shipping_limit_date','product_category_name', 'product_name_lenght', 
                                  'product_description_lenght', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm',
                                  'payment_installments','review_id', 'review_comment_title', 'review_comment_message', 'review_creation_date', 
                                  'review_answer_timestamp', 'customer_id'])

In [ ]:
df_olist_clean.head()

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_photos_qty,payment_sequential,payment_type,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1.0,a9516a079e37a9c9c36b9b78b10169e8,124.99,21.88,1.0,1.0,credit_card,146.87,4.0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,office_furniture
1,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,1.0,bd07b66896d6f1494f5b86251848ced7,139.94,17.79,1.0,1.0,credit_card,157.73,5.0,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,office_furniture
2,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1.0,a5647c44af977b148e0a3a4751a09e2e,149.94,23.36,1.0,1.0,credit_card,173.30,5.0,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,office_furniture
3,997fd4b37386d10b57d4ac8cdec011a6,2018-07-08 09:59:19,1.0,8ed094bfe076c568f6bb10feada3f75d,199.98,115.77,1.0,1.0,credit_card,315.75,1.0,07d190f123147d9e89d4b922543d7948,65075,sao luis,MA,office_furniture
4,b63fff1ef8b6201df68bb611ee41129e,2017-04-02 22:15:27,1.0,aba86c093ccdbac75b09111d57e50004,129.60,16.61,1.0,1.0,boleto,146.21,4.0,332cf4e83e16004ba7dca932ce82475b,90010,porto alegre,RS,office_furniture


## Attribute Information:
<table>
  <tr>
    <th>Attribute</th>
    <th>Data Type</th>
    <th>Description</th>
    <th>Unit Analysis</th>
  </tr>
  <tr>
    <td>order_id</td>
    <td>object</td>
    <td>Unik ID dari order</td>
    <td>Setiap baris merepresentasikan ID order yang yang muncul ketika customer
    melakukan order</td>
  </tr>
  <tr>
    <td>order_purchase_stamp</td>
    <td>datetime64[s]</td>
    <td>Waktu pembelian customer</td>
    <td>Setiap baris merepresentasikan waktu ketika customer melakukan order</td>
  </tr>
  <tr>
    <td>order_item_id</td>
    <td>float64</td>
    <td>Jumlah barang per-order</td>
    <td>Setiap baris merepresentasikan jumlah barang yang dibeli customer dalam sekali order</td>
  </tr>
  <tr>
    <td>product_id</td>
    <td>object</td>
    <td>ID dari sebuah product</td>
    <td>Setiap baris merepresentasikan ID dari sebuah product yang diorder oleh customer</td>
  </tr>
  <tr>
    <td>price</td>
    <td>float64</td>
    <td>Harga dari sebuah product</td>
    <td>Setiap baris merepresentasikan harga dari sebuah product yang diorder oleh customer</td>
  </tr>
  <tr>
    <td>freight_value</td>
    <td>float64</td>
    <td>Nilai pengiriman product</td>
    <td>Setiap baris merepresentasikan nilai pengirim dari sebuah product yang diorder oleh customer. Apabila item lebih dari 1, nilainya dipisah</td>
  </tr>
  <tr>
    <td>product_photo_qty</td>
    <td>float64</td>
    <td>Jumlah foto sebuah product</td>
    <td>Setiap baris merepresentasikan jumlah foto dari sebuah product yang diiklankan</td>
  </tr>
  <tr>
    <td>payment_sequential</td>
    <td>float64</td>
    <td>Jumlah payment method</td>
    <td>Setiap baris merepresentasikan jumlah metode payment yang dilakukan oleh customer. Customer dapat membayar sebuah transaksi lebih dari 1 metode</td>
  </tr>
  <tr>
    <td>payment_type</td>
    <td>object</td>
    <td>Metode pembayaran</td>
    <td>Setiap baris merepresentasikan metode yang digunakan customer dalam membayar sebuah transaksi</td>
  </tr>
  <tr>
    <td>payment_value</td>
    <td>float64</td>
    <td>Nilai Transaksi</td>
    <td>Setiap baris merepresentasikan nilai transaksi yang harus dibayar customer</td>
  </tr>
  <tr>
    <td>review_score</td>
    <td>float64</td>
    <td>Tingkat kepuasan customer</td>
    <td>Setiap baris merepresentasikan tingkat kepuasan customer dalam bertransaksi, nilainya dari 1 - 5</td>
  </tr>
  <tr>
    <td>customer_unique_id</td>
    <td>object</td>
    <td>Unik ID dari customer</td>
    <td>Setiap baris merepresentasikan ID customer yang melakukan
    order</td>
  </tr>
  <tr>
    <td>customer_zip_code_prefix</td>
    <td>int64</td>
    <td>Kode pos dari customer</td>
    <td>Setiap baris merepresentasikan kode pos dari tempat tinggal customer, namun hanya 5 kode awal</td>
  </tr>
  <tr>
    <td>customer_city</td>
    <td>object</td>
    <td>Nama kota dari customer</td>
    <td>Setiap baris merepresentasikan nama kota tempat tinggal customer</td>
  </tr>
  <tr>
    <td>customer_state</td>
    <td>object</td>
    <td>Negara Bagian customer</td>
    <td>Setiap baris merepresentasikan negara bagian dari tempat tinggal customer</td>
  </tr>
  <tr>
    <td>product_category_name_english</td>
    <td>object</td>
    <td>Kategori product</td>
    <td>Setiap baris merepresentasikan nama kategori dari sebuah product dalam Bahasa Inggris</td>
  </tr>
  <tr>
    <td>month_order</td>
    <td>object</td>
    <td>Nama bulan order dilakukan</td>
    <td>Setiap baris merepresentasikan nama bulan dari tanggal pembelian dilakukan</td>
  </tr>
  <tr>
    <td>weekday_order</td>
    <td>object</td>
    <td>Nama hari order dilakukan</td>
    <td>Setiap baris merepresentasikan nama hari dari tanggal pembelian dilakukan</td>
  </tr>
  <tr>
    <td>month_year_order</td>
    <td>period[M]</td>
    <td>Tahun dan bulan pembelian</td>
    <td>Setiap baris merepresentasikan tahun dan bulan dari tanggal pembelian dilakukan</td>
  </tr>

</table>

# Preprocessing Part 1:

### Handling Missing Value:

In [ ]:
df_olist_clean.isnull().sum()

order_id                         0
order_purchase_timestamp         0
order_item_id                    0
product_id                       0
price                            0
freight_value                    0
product_photos_qty               0
payment_sequential               3
payment_type                     3
payment_value                    3
review_score                     0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
product_category_name_english    0
dtype: int64

In [ ]:
df_olist_clean[df_olist_clean['payment_type'].isnull()]

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_photos_qty,payment_sequential,payment_type,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
51505,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,1.0,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,1.0,NaN,NaN,NaN,1.0,830d5b7aaa3b6f1e9ad63703bec97d23,14600,sao joaquim da barra,SP,health_beauty
51506,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,2.0,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,1.0,NaN,NaN,NaN,1.0,830d5b7aaa3b6f1e9ad63703bec97d23,14600,sao joaquim da barra,SP,health_beauty
51507,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,3.0,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,1.0,NaN,NaN,NaN,1.0,830d5b7aaa3b6f1e9ad63703bec97d23,14600,sao joaquim da barra,SP,health_beauty


Column "payment_sequential", "payment_type", dan "payment_value" terdapat 3 data yang *missing value*. *Missing value* pada column tersebut tergolong dalam *Missing Value Completely at Random (MCAR)* yang artinya *missing data* yang terjadi benar-benar secara *random* dan tidak berkaitan dengan variable lain maupun dirinya sendiri. Kemungkinan hal ini terjadi karena adanya kesalahan dalam meng-*input* data. Untuk mengatasi hal tersebut, kami memutuskan untuk men-*drop* ketiga data tersebut dikarenakan jumlah data yang kami punya juga cukup besar. 

In [ ]:
df_olist_clean.dropna(inplace = True)

In [ ]:
df_olist_clean.isnull().sum()

order_id                         0
order_purchase_timestamp         0
order_item_id                    0
product_id                       0
price                            0
freight_value                    0
product_photos_qty               0
payment_sequential               0
payment_type                     0
payment_value                    0
review_score                     0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
product_category_name_english    0
dtype: int64

### Handling Duplicate Value:
Proses selanjutnya adalah kita melihat apakah data yang sudah kita *clean* ada data yang *duplicate* atau tidak.

In [ ]:
df_olist_clean.duplicated().sum()

414

In [ ]:
df_olist_clean[df_olist_clean.duplicated()].head()

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_photos_qty,payment_sequential,payment_type,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
53,5040757d4e06a4be96d3827b860b4e7c,2017-10-20 23:02:08,1.0,f80214fd7d681d1dcf12b3acaf4f0da7,196.99,56.66,1.0,1.0,credit_card,253.65,5.0,8746f1ba2e19bcf8bc03a946a0923d78,83511,almirante tamandare,PR,office_furniture
1254,cb5b7fb63c0add7098c025cb737df02e,2018-02-25 17:37:04,1.0,2b7aa376a6e728560bddb5558cc48e89,116.99,17.26,1.0,1.0,boleto,134.25,5.0,a982bd0fec12d781fecd9f1e17cb81bf,48967,nucleo residencial pilar,BA,office_furniture
2706,712cb14ddeca38aaeb49dea61f811018,2018-02-27 15:22:01,1.0,a2a61ecd4e916c3ff787c137d23957bb,47.20,16.60,1.0,1.0,credit_card,63.80,5.0,3a5b0d10ee72cefb504df217532a4e6a,33400,lagoa santa,MG,housewares
2930,9de38c42ea4d33af19aac6c4d6c4b146,2018-06-12 16:12:29,1.0,259bd66612b61aa4e82ce3cde9272bbc,25.00,12.84,5.0,1.0,credit_card,37.84,5.0,68215d6b5740453cc6029d69bd017f68,12515,guaratingueta,SP,housewares
3383,cba44aa41766f6210b875f576a4241e4,2018-07-16 18:10:36,1.0,5d2d01b64b76916f0cbe74e407041187,36.40,15.36,1.0,1.0,credit_card,51.76,5.0,564df947eb9192d7e1613b1b957cb192,71680,brasilia,DF,housewares


Dari hasil diatas, ditemukan ada 414 data yang memiliki nilai yang benar-benar sama. Pada umumnya, *duplicate value* perlu dihapus, namun sebelum dihapus kami akan melakukan pengecekan pada data yang belum diproses untuk mengetahui apakah data yang *duplicate* ini memiliki arti atau tidak.

In [ ]:
df_olist[(df_olist['order_id'] == "5040757d4e06a4be96d3827b860b4e7c") | (df_olist['order_id'] == "cb5b7fb63c0add7098c025cb737df02e")]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
52,5040757d4e06a4be96d3827b860b4e7c,0db6ca6d0557c5b2358816e1ed450384,delivered,2017-10-20 23:02:08,2017-10-20 23:14:10,2017-10-27 22:22:26,2017-11-06 22:24:38,2017-11-21 00:00:00,1.0,f80214fd7d681d1dcf12b3acaf4f0da7,7c67e1448b00f6e969d365cea6b010ab,2017-10-31 23:14:10,196.99,56.66,moveis_escritorio,45.0,1156.0,1.0,12250.0,63.0,58.0,24.0,1.0,credit_card,10.0,253.65,afd9b3d196f0acd81d79d8ccd79c2430,5.0,NaN,NaN,2017-10-31 00:00:00,2017-11-04 21:12:29,8746f1ba2e19bcf8bc03a946a0923d78,83511,almirante tamandare,PR,office_furniture
53,5040757d4e06a4be96d3827b860b4e7c,0db6ca6d0557c5b2358816e1ed450384,delivered,2017-10-20 23:02:08,2017-10-20 23:14:10,2017-10-27 22:22:26,2017-11-06 22:24:38,2017-11-21 00:00:00,1.0,f80214fd7d681d1dcf12b3acaf4f0da7,7c67e1448b00f6e969d365cea6b010ab,2017-10-31 23:14:10,196.99,56.66,moveis_escritorio,45.0,1156.0,1.0,12250.0,63.0,58.0,24.0,1.0,credit_card,10.0,253.65,46abf3ea0b2710ad41390fdb79c32d84,5.0,NaN,NaN,2017-11-07 00:00:00,2017-11-10 20:07:48,8746f1ba2e19bcf8bc03a946a0923d78,83511,almirante tamandare,PR,office_furniture
1253,cb5b7fb63c0add7098c025cb737df02e,7e2d8e10c5c9339ad1492ad76452465f,delivered,2018-02-25 17:37:04,2018-02-27 04:31:06,2018-03-01 21:36:54,2018-03-19 16:46:35,2018-04-12 00:00:00,1.0,2b7aa376a6e728560bddb5558cc48e89,7c67e1448b00f6e969d365cea6b010ab,2018-03-13 03:51:50,116.99,17.26,moveis_escritorio,59.0,1296.0,1.0,13388.0,54.0,47.0,28.0,1.0,boleto,1.0,134.25,167b40b617e650de2c473d70baae2e65,5.0,NaN,entrega no prazo correto produto em otima qual...,2018-03-16 00:00:00,2018-03-19 16:37:49,a982bd0fec12d781fecd9f1e17cb81bf,48967,nucleo residencial pilar,BA,office_furniture
1254,cb5b7fb63c0add7098c025cb737df02e,7e2d8e10c5c9339ad1492ad76452465f,delivered,2018-02-25 17:37:04,2018-02-27 04:31:06,2018-03-01 21:36:54,2018-03-19 16:46:35,2018-04-12 00:00:00,1.0,2b7aa376a6e728560bddb5558cc48e89,7c67e1448b00f6e969d365cea6b010ab,2018-03-13 03:51:50,116.99,17.26,moveis_escritorio,59.0,1296.0,1.0,13388.0,54.0,47.0,28.0,1.0,boleto,1.0,134.25,a334abcb91df2132f817e0ae76ee5d55,5.0,NaN,produto em perfeito de otima qualidade e em pe...,2018-03-20 00:00:00,2018-03-23 15:59:14,a982bd0fec12d781fecd9f1e17cb81bf,48967,nucleo residencial pilar,BA,office_furniture


Hasil pengecekan menunjukan bahwa data yang *duplicate* terjadi karena adanya *customer* yang memberikan 2 *review* terhadap transaksi yang sama. Berdasarkan pengecekan tersebut, kami memutuskan untuk menghapus data yang *duplicate* karena transaksi yang terjadi sebenarnya sama.

In [ ]:
df_olist_clean = df_olist_clean.drop_duplicates()

### Casting Data Types dan Feature Engineering:
*Formating Data Types* dilakukan untuk memastikan bahwa *data types* yang kami punya sesuai dengan yang seharusnya. Apabila tidak sesuai, maka *data types* dari sebuah *columns* harus diubah. Selain itu, *Formatting Data Types* juga digunakan untuk memastikan bahwa value dari *data categorical* sudah konsisten, tidak ada nama yang memiliki arti yang sama, namun *categorical*-nya berbeda

In [ ]:
# Cek tipe data tiap column
df_olist_clean.dtypes

order_id                          object
order_purchase_timestamp          object
order_item_id                    float64
product_id                        object
price                            float64
freight_value                    float64
product_photos_qty               float64
payment_sequential               float64
payment_type                      object
payment_value                    float64
review_score                     float64
customer_unique_id                object
customer_zip_code_prefix           int64
customer_city                     object
customer_state                    object
product_category_name_english     object
dtype: object

Dari hasil pengecekan, *columns* "order_purchase_timestamp" akan lebih baik apabila *data types*-nya diubah menjadi *datetime*. Selain mengubah *data types*, kami juga menambahkan beberapa *column* baru dari hasil ekstraksi *column* "order_purchase_timestamp" yang kemungkinan dapat digunakan pada proses *EDA*

In [ ]:
# Convert type data tanggal jadi datetime
df_olist_clean['order_purchase_timestamp'] = pd.to_datetime(df_olist_clean['order_purchase_timestamp'])
df_olist_clean['order_purchase_timestamp'].dt.strftime('%Y-%m-%d')

df_olist_clean['month_order'] = df_olist_clean['order_purchase_timestamp'].dt.month_name()
df_olist_clean['weekday_order'] = df_olist_clean['order_purchase_timestamp'].dt.day_name()
df_olist_clean['month_year_order'] = df_olist_clean['order_purchase_timestamp'].dt.to_period('M')
df_olist_clean['date_order'] = df_olist_clean['order_purchase_timestamp'].dt.day

df_olist_clean.head()

,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_photos_qty,payment_sequential,payment_type,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english,month_order,weekday_order,month_year_order,date_order
0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1.0,a9516a079e37a9c9c36b9b78b10169e8,124.99,21.88,1.0,1.0,credit_card,146.87,4.0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,office_furniture,May,Tuesday,2017-05,16
1,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,1.0,bd07b66896d6f1494f5b86251848ced7,139.94,17.79,1.0,1.0,credit_card,157.73,5.0,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,office_furniture,May,Saturday,2018-05,19
2,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1.0,a5647c44af977b148e0a3a4751a09e2e,149.94,23.36,1.0,1.0,credit_card,173.30,5.0,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,office_furniture,March,Tuesday,2018-03,13
3,997fd4b37386d10b57d4ac8cdec011a6,2018-07-08 09:59:19,1.0,8ed094bfe076c568f6bb10feada3f75d,199.98,115.77,1.0,1.0,credit_card,315.75,1.0,07d190f123147d9e89d4b922543d7948,65075,sao luis,MA,office_furniture,July,Sunday,2018-07,8
4,b63fff1ef8b6201df68bb611ee41129e,2017-04-02 22:15:27,1.0,aba86c093ccdbac75b09111d57e50004,129.60,16.61,1.0,1.0,boleto,146.21,4.0,332cf4e83e16004ba7dca932ce82475b,90010,porto alegre,RS,office_furniture,April,Sunday,2017-04,2


In [ ]:
df_olist_clean.dtypes

order_id                                 object
order_purchase_timestamp         datetime64[ns]
order_item_id                           float64
product_id                               object
price                                   float64
freight_value                           float64
product_photos_qty                      float64
payment_sequential                      float64
payment_type                             object
payment_value                           float64
review_score                            float64
customer_unique_id                       object
customer_zip_code_prefix                  int64
customer_city                            object
customer_state                           object
product_category_name_english            object
month_order                              object
weekday_order                            object
month_year_order                      period[M]
date_order                                int64
dtype: object

### Inconsisten Variable:
Untuk memastikan bahwa data categorical nilainya konsisten, kami akan mengecek data-data categorical yang kami punya. Data Categorical yang kami punya adalah "payment_type", "customer_city", "customer_state", "product_category_name_english"

In [ ]:
# Cek categorical data
categorical_data = ["payment_type", "customer_city", "customer_state", "product_category_name_english"]

for i in categorical_data:
  print(f"Unique Value dari {i}")
  print(sorted(df_olist_clean[i].unique()))
  print('\n')

Unique Value dari payment_type
['boleto', 'credit_card', 'debit_card', 'voucher']


Unique Value dari customer_city
['abadia dos dourados', 'abadiania', 'abaete', 'abaetetuba', 'abaiara', 'abaira', 'abare', 'abatia', 'abdon batista', 'abelardo luz', 'abrantes', 'abre campo', 'abreu e lima', 'acaiaca', 'acailandia', 'acajutiba', 'acarau', 'acari', 'acegua', 'acopiara', 'acreuna', 'acu', 'acucena', 'adamantina', 'adhemar de barros', 'adolfo', 'adrianopolis', 'adustina', 'afogados da ingazeira', 'afonso claudio', 'afranio', 'agisse', 'agrolandia', 'agronomica', 'agua boa', 'agua branca', 'agua clara', 'agua comprida', 'agua doce', 'agua doce do norte', 'agua fria de goias', 'agua limpa', 'agua nova', 'agua preta', 'agua santa', 'aguai', 'aguas belas', 'aguas claras', 'aguas da prata', 'aguas de lindoia', 'aguas de santa barbara', 'aguas de sao pedro', 'aguas formosas', 'aguas frias', 'aguas lindas de goias', 'aguas mornas', 'aguas vermelhas', 'agudo', 'agudos', 'aguia branca', 'aimores', 

Bila diperhatikan, pada column "product_category_name_english" ada nilai yang memiliki 2 versi penamaan, seperti "home_appliances" dan "home_appliances_2", "home_confort" dan "home_comfort_2" sehingga kedua kategori tersebut akan dibuat sama.

In [ ]:
def format_val(x):
  if x == 'home_appliances_2':
    return "home_appliances"
  elif x == 'home_confort':
    return "home_comfort"
  elif x == "home_comfort_2":
    return "home_comfort"
  
  return x

df_olist_clean['product_category_name_english'] = df_olist_clean['product_category_name_english'].apply(format_val)

In [ ]:
print("Unique value dari product_category_name_english:\n")
print(sorted(df_olist_clean['product_category_name_english'].unique()))

Unique value dari product_category_name_english:

['agro_industry_and_commerce', 'air_conditioning', 'art', 'arts_and_craftmanship', 'audio', 'auto', 'baby', 'bed_bath_table', 'books_general_interest', 'books_imported', 'books_technical', 'cds_dvds_musicals', 'christmas_supplies', 'cine_photo', 'computers', 'computers_accessories', 'consoles_games', 'construction_tools_construction', 'construction_tools_lights', 'construction_tools_safety', 'cool_stuff', 'costruction_tools_garden', 'costruction_tools_tools', 'diapers_and_hygiene', 'drinks', 'dvds_blu_ray', 'electronics', 'fashio_female_clothing', 'fashion_bags_accessories', 'fashion_childrens_clothes', 'fashion_male_clothing', 'fashion_shoes', 'fashion_sport', 'fashion_underwear_beach', 'fixed_telephony', 'flowers', 'food', 'food_drink', 'furniture_bedroom', 'furniture_decor', 'furniture_living_room', 'furniture_mattress_and_upholstery', 'garden_tools', 'health_beauty', 'home_appliances', 'home_comfort', 'home_construction', 'houseware

# Save Data:
Setelah proses membersihkan data yang missing dan duplicate, kami akan menyimpan data tersebut agar dapat digunakan kembali. Pembersihan outlier akan dilakukan pada step EDA karena untuk mendeteksi outlier kami perlu melakukan explorasi datanya terlebih dahulu agar kami tidak sembarangan menghapus outlier

In [ ]:
df_olist_clean.to_csv('/content/drive/MyDrive/Project Purwadhika/data_outlist_clean.csv', index = False, date_format = '%Y-%m-%d')